In the second part of this assignment, we will implement support vector machine and kernel support vector machine.

## **Preparation**
The following code is the preparation for importing packages. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cvxopt as cvx

## **[Step 1-3] Soft margin support vector machine (SVM) implementation**

In this assignment, you should use only NumPy to build the SVM models.

**DO NOT** use other libraries to implement the SVM models.

**DO NOT** modify other parts of the skeleton code.

Follow the comments. They'll give you instructions on what to code.

### Step 1. Build the soft margin SVM models based on the gradient descent
The followings are the skeleton codes. Implement the functions inside the ```SVM``` class.

In [ ]:
class SVM():
    def __init__(self, X_data, reg=1):
        # Initialize the model parameters as the instance variable.
        print("-"*50, "\nInitialize the parameters of our SVM.\n")
        self.W = np.random.randn(X_data.shape[1])
        self.w_0 = np.random.randn(1)
        self.reg = reg
    
    # Function for calculating the loss.
    def objective_function(self, X_data, y_label):
        # Define the objective function for the SVM classifier.
        distance = 1 - y_label * (X_data.dot(self.W) + self.w_0)
        distance[distance < 0] = 0.
        cost = 1/2 * self.reg * self.W.T.dot(self.W) + np.sum(distance, axis=0) / X_data.shape[0]
        return cost
    
    # Function for calculating the gradient 
    def calculate_gradient(self, X_data, y_label):
        # Compute the gradient of W and w_0 from the entire epoch.
        distance = np.sign(1 - y_label * (X_data.dot(self.W) + self.w_0))
        distance[distance < 0] = 0.
        gradient_W = self.reg * self.W - np.sum((X_data.T * y_label * distance).T, axis=0) / X_data.shape[0]
        gradient_w_0 = -np.sum(y_label * distance, axis=0) / X_data.shape[0]
                
        return gradient_W, gradient_w_0
    
    # Function for training
    def train(self, x_input, y_label, lr=0.01, epochs=50):
        # Update the model parameters (W, w_0) using the above calculate_gradient function.
        print("- Total Epochs \t\t: ", epochs)
        print("- Learning rate \t: ", lr)
        print("-"*50)
        for epoch in range(epochs):
            cost = self.objective_function(x_input, y_label)
            gradient_W, gradient_w_0 = self.calculate_gradient(x_input, y_label)
            self.W += -1 * lr * gradient_W
            self.w_0 += -1 * lr * gradient_w_0
            if epoch % 10 == 0:
                print("[Epoch : %d/%d]\t\t| Cost: %.4f"%(epoch+10, epochs, cost))
        print("-"*50, "\nLearning is complete.\n")
        print("- W \t:", self.W)
        print("- W_0 \t:", self.w_0)

    # Function for prediction
    def predict(self, x_input):
        # Compute our SVM prediction, and transform it into the binary label.
        result = np.sign(x_input.dot(self.W) + self.w_0)
        return result
    
    # Function for evaluation
    def accuracy(self, predict, y_label):
        result = np.sum(predict == y_label) / len(predict)
        return result

### Step 2. Load the dataset 1
The dataset is the isotropic Gaussian blobs with noise for classification.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load the train, validation, test dataset.
train = np.load("./drive/MyDrive/Dataset1(Train).npy")
validation = np.load("./drive/MyDrive/Dataset1(Validation).npy")
test = np.load("./drive/MyDrive/Dataset1(Test).npy")

# Set the X, y data for each dataset. 
X_train = train[:, :-1]
y_train = train[:, -1]

X_val = validation[:, :-1]
y_val = validation[:, -1]

X_test = test[:, :-1]
y_test = test[:, -1]

# Visualize the train dataset.
fig = plt.figure(figsize=(9, 6), dpi=100)
ax = fig.add_subplot(111, projection='3d')
ax.plot(X_train[:, 0][y_train == -1], X_train[:, 1][y_train == -1], X_train[:, 2][y_train == -1], '.', label="Class 1")
ax.plot(X_train[:, 0][y_train == 1], X_train[:, 1][y_train == 1], X_train[:, 2][y_train == 1], '.', label="Class 2")
plt.title("The 3D plot of the train dataset", fontsize=17)
ax.set_xlabel('Feature 1')
ax.set_ylabel('Feature 2')
ax.set_zlabel('Feature 3')
ax.grid()
ax.legend()
plt.show()

In [ ]:
# Check the size of each dataset.
print("The shape of the train dataset\t\t: %s\t | The shape of train label\t\t: %s"%(X_train.shape, y_train.shape))
print("The shape of the validation dataset\t: %s\t | The shape of validation label\t: %s"%(X_val.shape, y_val.shape))
print("The shape of the test dataset\t\t: %s\t | The shape of test label\t\t: %s"%(X_test.shape, y_test.shape))

### Step 3. Train and test the soft margin SVM models with different learning rates.
Check the tendency of the loss value that changes with the different learning rates. Visualize the result of the SVM models with the different learning rates.

In [ ]:
np.random.seed(111)

# Set variables(hyperparameters) to train our SVM model.
lr = 1e-4
epochs = 100
reg = 1

train_setting = {"lr": [1e-2, 1e-3]}
models = []
for i, lr in enumerate(train_setting["lr"]):
    # Build our SVM model.
    model = SVM(X_train, reg)
    # Start train our SVM model.
    model.train(X_train, y_train, lr=lr, epochs=epochs)

    # Predict the results using our trained SVM model.
    train_predicted = model.predict(X_train)
    # Compute the accuracy of prediction for the train set
    acc = model.accuracy(train_predicted, y_train)
    print("\n- The accuracy of the train set \t: %.3f"%acc)

    val_predicted = model.predict(X_val)
    acc = model.accuracy(val_predicted, y_val)
    print("- The accuracy of the validation set \t: %.3f"%acc)
    print("-"*50,"\n")
    models.append((model, lr))

#### Visualize a trained SVM model (idx-th) with the 3D plot

In [ ]:
# Visualize a trained SVM model (idx-th) with the 3D plot
idx = 0
model = models[idx]

test_predicted = model[0].predict(X_test)
acc = model[0].accuracy(test_predicted, y_test)
print("- The accuracy of prediction for the test set \t:", acc)

# Set the axis to generate the meshgrid axis
x_axis = np.arange(-50, 100, 1)
y_axis = np.arange(-50, 100, 1)
x_axis, y_axis = np.meshgrid(x_axis, y_axis)

# Define the hyper plane, and then visualize it. 
Z = - (x_axis * model[0].W[0] + y_axis * model[0].W[1] + model[0].w_0)/model[0].W[2]
fig = plt.figure(figsize=(9, 6), dpi=100)
ax = fig.add_subplot(111, projection='3d')

# Plot the linearly separable sample datasets with the predicted labels.
ax.plot(X_test[:, 0][y_test == -1], X_test[:, 1][y_test == -1], X_test[:, 2][y_test == -1], '.', label="Class 1")
ax.plot(X_test[:, 0][y_test == 1], X_test[:, 1][y_test == 1], X_test[:, 2][y_test == 1], '.', label="Class 2")
label = "Hyperplane"
plane = ax.plot_surface(x_axis, y_axis, Z, label = label)
plane._facecolors2d = plane._facecolor3d
plane._edgecolors2d = plane._edgecolor3d

ax.set_xlabel('Feature 1')
ax.set_ylabel('Feature 2')
ax.set_zlabel('Feature 3')
ax.view_init(25,-90)
ax.grid()
ax.legend(loc='lower left')
plt.title("Test result of the SVM classification with lr=0.01", fontsize=15)
plt.show()

In [ ]:
# Visualize a trained SVM model (idx-th) with the 3D plot
idx = 1
model = models[idx]

test_predicted = model[0].predict(X_test)
acc = model[0].accuracy(test_predicted, y_test)
print("- The accuracy of prediction for the test set \t:",acc)

# Set the axis to generate the meshgrid axis
x_axis = np.arange(-50, 100, 1)
y_axis = np.arange(-50, 100, 1)
x_axis, y_axis = np.meshgrid(x_axis, y_axis)

# Define the hyper plane, and then visualize it. 
Z = -(x_axis * model[0].W[0] + y_axis * model[0].W[1] + model[0].w_0) / model[0].W[2]
fig = plt.figure(figsize=(9, 6), dpi=100)
ax = fig.add_subplot(111, projection='3d')

# Plot the linearly separable sample datasets with the predicted labels.
ax.plot(X_test[:, 0][y_test == -1], X_test[:, 1][y_test == -1], X_test[:, 2][y_test == -1], '.', label="Class 1")
ax.plot(X_test[:, 0][y_test == 1], X_test[:, 1][y_test == 1], X_test[:, 2][y_test == 1], '.', label="Class 2")
label = "Hyperplane"
plane = ax.plot_surface(x_axis, y_axis, Z, label = label)
plane._facecolors2d = plane._facecolor3d
plane._edgecolors2d = plane._edgecolor3d

ax.set_xlabel('Feature 1')
ax.set_ylabel('Feature 2')
ax.yaxis.set_ticklabels([])
ax.set_zlabel('Feature 3')
ax.view_init(1,90)
ax.grid()
ax.legend(loc='lower right')
plt.title("Test result of the SVM classification with lr=0.001", fontsize=15)
plt.show()

## **[Step 4-6] Kernel SVM implementation**

In this assignment, you should use only NumPy and CVXOPT to build the Kernel SVM models.

**DO NOT** use other libraries to implement the Kernel SVM models.

**DO NOT** modify other parts of the skeleton code.

Follow the comments. They'll give you instructions on what to code.

### Step 4. Build the Kernel SVM models based on the dual form
The followings are the skeleton codes. Implement the functions inside the ```kernels``` class and the ```Kernel_SVM```.

In [ ]:
class kernels: 
    # Function for computing the kernel matrix
    def __init__(self, poly_c=10, poly_d=2, gamma=0.001):
        self.poly_c = poly_c
        self.poly_d = poly_d
        self.gamma = gamma
    
    # Function for the polynomial kernel.
    def polynomial_kernel(self, x, y, c=10, d=2):
        # Implement the polynomial kernel function
        result = (c + x.dot(y)) ** d
        return result
    
    # Function for the gaussian kernel.
    def gaussian_kernel(self, x, y, gamma=0.001):
        # Implement the linear kernel function.
        # Gamma means 1/(2*sigma**2) where sigma is a free parameter.
        # It can be seen as the standard deviation of the gaussian distribution.
        result = np.linalg.norm(np.array(x)-np.array(y), ord=None)**2
        result = np.exp(-gamma*result)
        return result

# Function for computing the kernel matrix
def compute_inner_matrix(X_data, kernel):
    # Generate the kernel matrix using the kernel function.
    n = X_data.shape[0]
    inner_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            inner_matrix[i,j] = kernel(X_data[i], X_data[j])
    return inner_matrix

In [ ]:
class Kernel_SVM:
    def __init__(self, kernel, threshold_SV):
        # Set the parameters as the hinstance variables.
        self.kernel = kernel
        self.threshold_SV = threshold_SV
    
    # Function for computing the lagrangian multiplier vector.
    def compute_lagrange(self, X_data, y_label):
        #Here we will use the cvxopt python library. We will use its matrix objects cvx.matrix
        # Compute the kernel matrix using kernel function.
        kernel_matrix = compute_inner_matrix(X_data, self.kernel)
        
        # Compute P matrix using y_label vectors and kernel_matrix.
        P = cvx.matrix(np.outer(y_label, y_label) * kernel_matrix, tc='d')
        # Set ones vector to compute the sum of the alpha vector.
        q = cvx.matrix(-np.ones(X_data.shape[0]), tc='d')
        
        # Set the identity metrix for our inequality constraint 
        # that all elements of the alpha vector should be greater then equal to zero.
        G = cvx.matrix(-np.identity(X_data.shape[0]), tc='d')
        h = cvx.matrix(np.zeros(X_data.shape[0]), tc='d')
        
        # Set the y_label vector for our equality constraint
        # that tha result of the multiplication of elements of the alpha and y_label vector should be zero. 
        A = cvx.matrix(y_label, (1, X_data.shape[0]), tc='d') 
        b = cvx.matrix(0.)
        
        # Compute the alpha vector.
        alpha = np.array(cvx.solvers.qp(P, q, G, h, A, b)['x']).flatten()

        return alpha

    # Function for training.
    def train(self,X_data, y_label):
        # Compute the alpha vector using compute_lagrange function.
        self.alpha = self.compute_lagrange(X_data, y_label)
        
        # Set the alpha values that are greater than the threshold.
        self.alpha_support = self.alpha[self.alpha > self.threshold_SV]
        # Set the support vectors corresponding to the value of alpha that is greater than the threshold.
        self.support_vectors = X_data[np.argwhere(self.alpha > self.threshold_SV)].reshape(-1, X_data.shape[1])
        # Set the labels of the support vectors corresponding to the alpha value that is greater than the threshold.
        self.support_vectors_labels = y_label[np.argwhere(self.alpha > self.threshold_SV)]

    # Function for prediction.
    def predict(self,x_data):
        # Compute our kernel SVM prediction, and transform it to the binary label.
        # And then append it to the list.
        y_pred = [] # It represents the predicted label of each data point,
        pred = [] # it represents the output value of each data point. (without sign function)
        for i in range(x_data.shape[0]):
            # Comput the bias vector to predict the label of each data point.
            self.bias = 0
            for (alpha, x_, y_) in zip(self.alpha_support, self.support_vectors, self.support_vectors_labels):
                # comput the output value of kernel SVM model.
                self.bias += y_
                self.bias = self.bias - np.sum(alpha * y_ * self.kernel(x_, x_data[i]))
            self.bias /= len(self.alpha_support)
            output = self.bias
            for (alpha, x_, y_) in zip(self.alpha_support, self.support_vectors, self.support_vectors_labels):
                # Compute the output value of our prediction 
                output += alpha * y_ * self.kernel(x_, x_data[i])
            # Using sign function, transform our prediction to the binary label. 
            y_pred.extend(np.sign(output))
            pred.extend(output)
        
        return y_pred, pred
    
    # Function for evaluation.
    def accuracy(self, predict, y_label):
        result = np.sum(predict == y_label) / len(predict)
        return result

### Step 5. Load the dataset 2
The dataset is the  isotropic Gaussian blobs with noise for classification. Load the dataset and split it into the train and test set by executing the provided code.

In [ ]:
# Load the train, validation, test dataset.
train = np.load("./drive/MyDrive/Dataset2(Train).npy")
validation = np.load("./drive/MyDrive/Dataset2(Validation).npy")
test = np.load("./drive/MyDrive/Dataset2(Test).npy")

# Set the X, y data for each dataset. 
X_train = train[:, :-1]
y_train = train[:, -1]
X_val = validation[:, :-1]
y_val = validation[:, -1]
X_test = test[:, :-1]
y_test = test[:, -1]

# Visualize the train dataset.
fig = plt.figure(figsize=(9, 6), dpi=100)
ax = fig.add_subplot(111)
ax.scatter(X_train[y_train == -1, 0], X_train[y_train == -1, 1], label="Class 1") 
ax.scatter(X_train[y_train == 1, 0], X_train[y_train == 1, 1], label="Class 2") 
ax.set_xlabel('Feature 1')
ax.set_ylabel('Feature 2')
plt.title("The 2D plot of the non-linearly separable sample dataset", fontsize=17)
plt.grid()
plt.legend()
plt.show()

In [ ]:
# Check the size of each dataset.
print("The shape of the train dataset\t\t: %s\t | The shape of train label\t\t: %s"%(X_train.shape, y_train.shape))
print("The shape of the validation dataset\t: %s\t | The shape of validation label\t: %s"%(X_val.shape, y_val.shape))
print("The shape of the test dataset\t\t: %s\t | The shape of test label\t\t: %s"%(X_test.shape, y_test.shape))

### Step 6. Train and test the Kernel SVM models with different types of kernels
Check the accuracy of the Kernel SVM model with different types of kernels.

- The performance of **the Polynomial Kernel SVM models** varies depending on the kernel’s hyperparameters ```poly_c``` and ```poly_d```.
- The performance of **the Gaussian Kernel SVM models** varies depending on the kernel’s hyperparameter ```gamma```.

#### Train the Gaussian Kernel SVM model with the different hyperparameters.

In [ ]:
# Feel free to experiment with hyperparameters
# Default values of the hyperparametes are as belows:
gamma = 0.001

utils_ = kernels(gamma=gamma)
kernel = utils_.gaussian_kernel

# Initialize the kernel SVM model, and then train it.
Gaussian_kernel = Kernel_SVM(kernel=kernel, threshold_SV=10**(-3))
Gaussian_kernel.train(X_train, y_train)

#### Evaluate the Gaussian Kernel SVM model

In [ ]:
# Predict the results using our trained SVM model.
train_predicted, pred = Gaussian_kernel.predict(X_train)
# Compute the accuracy of prediction for the train set
acc = Gaussian_kernel.accuracy(train_predicted, y_train)
print("- The accuracy of prediction for the train set \t\t:",acc)

val_predicted, pred = Gaussian_kernel.predict(X_val)
acc = Gaussian_kernel.accuracy(val_predicted, y_val)
print("- The accuracy of prediction for the validation set \t:",acc)

#### Visualize the result of the Gaussian Kernel SVM classification

In [ ]:
from matplotlib import cm
Z, _ = Gaussian_kernel.predict(X_test)
acc = Gaussian_kernel.accuracy(Z, y_test)
print("- The accuracy of prediction for the test set \t:",acc)
Z = np.array(Z)
fig = plt.figure(figsize=(9, 6), dpi=100)
ax = plt.axes(projection='3d')
ax.scatter(X_test[y_test == -1, 0], X_test[y_test == -1, 1], Z[y_test == -1], label="Class 1") 
ax.scatter(X_test[y_test == 1, 0], X_test[y_test == 1, 1], Z[y_test == 1], label="Class 2")

plt.title("The 3D plot of the test result of the kernel SVM classification\n(Gaussian kernel: $\gamma$ = 0.001)", fontsize=13)
ax.set_xlabel('Feature 1')
ax.set_ylabel('Feature 2')
ax.set_zlabel('Class label')
ax.view_init(20,10)
ax.grid()
ax.legend()
plt.show()

#### Train the Polynomial Kernel SVM model with the different hyperparameters.

In [ ]:
# Chose which kernel to use.
# Feel free to experiment with hyperparameters
# Default values of the hyperparametes are as belows:
poly_c = 10
poly_d = 2

utils_ = kernels(poly_c=poly_c, poly_d=poly_d)
kernel = utils_.polynomial_kernel

# Initialize the kernel SVM model, and then train it.
Polynomial_kernel = Kernel_SVM(kernel=kernel, threshold_SV=10**(-10))
Polynomial_kernel.train(X_train, y_train)

#### Evaluate the Polynomial Kernel SVM model

In [ ]:
# Predict the results of the polynomial kernel SVM model.
train_predicted, pred = Polynomial_kernel.predict(X_train)
# Compute the accuracy of prediction for the train set
acc = Polynomial_kernel.accuracy(train_predicted, y_train)
print("- The accuracy of prediction for the train set \t\t:",acc)

val_predicted, pred = Polynomial_kernel.predict(X_val)
acc = Polynomial_kernel.accuracy(val_predicted, y_val)
print("- The accuracy of prediction for the validation set \t:",acc)

#### Visualize the result of the Polynomial Kernel SVM classification

In [ ]:
Z, _ = Polynomial_kernel.predict(X_test)
acc = Polynomial_kernel.accuracy(Z, y_test)
print("- The accuracy of prediction for the test set \t:",acc)
Z = np.array(Z)
fig = plt.figure(figsize=(9, 6), dpi=100)
ax = plt.axes(projection='3d')
ax.scatter(X_test[y_test == -1, 0], X_test[y_test == -1, 1], Z[y_test == -1], label="Class 1") 
ax.scatter(X_test[y_test == 1, 0], X_test[y_test == 1, 1], Z[y_test == 1], label="Class 2")

plt.title("The 3D plot of the test result of the kernel SVM classification\n(Polynomial kernel: c = 10, d = 2)", fontsize=13)
ax.set_xlabel('Feature 1')
ax.set_ylabel('Feature 2')
ax.set_zlabel('Class label')
ax.view_init(20, 10)
ax.grid()
ax.legend()
plt.show()